In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import csv

In [ ]:
train = pd.read_csv('/kaggle/input/amazon-ml-challenge-2021-hackerearth/train.csv', escapechar = "\\", quoting = csv.QUOTE_NONE)
test = pd.read_csv('/kaggle/input/amazon-ml-challenge-2021-hackerearth/test.csv', escapechar = "\\", quoting = csv.QUOTE_NONE)
sample = pd.read_csv('/kaggle/input/amazon-ml-challenge-2021-hackerearth/sample_submission.csv')

In [ ]:
train.shape, test.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

In [ ]:
train.fillna(" ", inplace=True)
test.fillna(" ", inplace=True)

In [ ]:
train['CORPUS'] = train['TITLE'] + " " + train['BRAND']
test['CORPUS'] = test['TITLE'] + " " + test['BRAND']

In [ ]:
# train = train.sample(frac=0.01, replace=True, random_state=2021)
checks = pd.DataFrame(train['BROWSE_NODE_ID'].value_counts().sort_values(ascending=True)).reset_index()
checks.columns = ['Class','Counts']
checks.head()

In [ ]:
from nltk.corpus import wordnet

def get_synonyms(word):
    
    synonyms = set()
    
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym) 
    if word in synonyms:
        synonyms.remove(word)
    
    return list(synonyms)

In [ ]:
import random
from nltk.corpus import stopwords
stop_words = []
for w in stopwords.words('english'):
    stop_words.append(w)
print(stop_words)

In [ ]:
def synonym_replacement(words, n):
    
    words = words.split()
    
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        
        if num_replaced >= n: #only replace up to n words
            break

    sentence = ' '.join(new_words)

    return sentence

In [ ]:
from tqdm import tqdm
data_aug = pd.DataFrame()
# data_aug.columns=['CORPUS','BROWSE_NODE_ID']
for i in tqdm(checks.index):
#     if checks['Counts'][i] >= 100:
#         data_sampled = data[data['BROWSE_NODE_ID']==checks['Class'][i]].sample(n=100)[['CORPUS','BROWSE_NODE_ID']]
#         data_aug = pd.concat([data_aug, data_sampled], ignore_index=True)
#     else:
    for n in range(0,3):
        trial_sent = train[train['BROWSE_NODE_ID']==checks['Class'][i]].sample(n=1)['CORPUS'].values[0]
        data_sampled = pd.DataFrame([synonym_replacement(trial_sent,n),checks['Class'][i]]).T
        data_sampled.columns=['CORPUS','BROWSE_NODE_ID']
        data_aug = pd.concat([data_aug,data_sampled],ignore_index=True)
data_aug.head()

In [ ]:
# # train = train.sample(n=100000)
# trg = list(checks[checks['Counts'] >= 2]['Class'].values)
# train = train[train['BROWSE_NODE_ID'].isin(trg)]

In [ ]:
# len(trg), train.shape

In [ ]:
# checks = pd.DataFrame(train['BROWSE_NODE_ID'].value_counts().sort_values(ascending=True)).reset_index()
# checks.columns = ['Class','Counts']
# checks.head()

In [ ]:
cv = CountVectorizer(stop_words='english', max_features=100)
train_feats = cv.fit_transform(data_aug['CORPUS']).toarray()
test_feats = cv.transform(test['CORPUS']).toarray()

In [ ]:
target = data_aug['BROWSE_NODE_ID']
target= target.astype('int')
test_id = test['PRODUCT_ID']

In [ ]:
data_aug['BROWSE_NODE_ID'].value_counts()

In [ ]:
data_aug.shape

In [ ]:
data_aug.head()

In [ ]:
del train
del data_aug
del test

In [ ]:
""" classifier """

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
# lr_clf = Pipeline([('cv', CountVectorizer(stop_words='english', max_features=50)),
#                    ('clf', ExtraTreesClassifier(class_weight='balanced'))
#                   ])
lr_clf = LogisticRegression(class_weight='balanced', verbose=True, n_jobs=-1)
lr_clf.fit(train_feats, target)

In [ ]:
predicted = lr_clf.predict(test_feats)

In [ ]:
len(predicted)

In [ ]:
# test = pd.read_csv('/kaggle/input/amazon-ml-challenge-2021-hackerearth/test.csv', escapechar = "\\", quoting = csv.QUOTE_NONE)

In [ ]:
result = pd.DataFrame()
result['PRODUCT_ID'] = test_id
result['BROWSE_NODE_ID'] = predicted
result.to_csv('lr_cv_v1.csv', index=False)

In [ ]:
result['BROWSE_NODE_ID'].value_counts()